<a href="https://colab.research.google.com/github/exusia1/Cyberbullying_hw/blob/main/Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
import re
from tensorflow.keras.layers import Dense, SimpleRNN, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from google.colab import files
from io import BytesIO
from PIL import Image
from google.colab import files

In [2]:
#files.upload()
with open('/content/voyna-i-mir-tom-1.txt', 'r', encoding='windows-1251') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) # заменяем все символы кроме кириллицы на пустые символы

In [3]:
# парсим текст, как последовательность символов
num_characters = 34 # 33 буквы + пробел
tokenizer = Tokenizer(num_words=num_characters, char_level=True) # токенизируем на уровне символов
tokenizer.fit_on_texts([text]) # формируем токены на основе частотности внашем тексте
print(tokenizer.word_index)
inp_chars = 6
data = tokenizer.texts_to_matrix(text) # преобразуем исходный текст в массив OHE
n = data.shape[0] - inp_chars # так как мы предсказываем по трем символам - четвертый
X = np.array([data[i:i + inp_chars, :] for i in range(n)])
Y = data[inp_chars:] # предсказание следующего символа
print(data.shape)

{' ': 1, 'о': 2, 'а': 3, 'е': 4, 'и': 5, 'н': 6, 'т': 7, 'с': 8, 'л': 9, 'в': 10, 'р': 11, 'к': 12, 'д': 13, 'м': 14, 'у': 15, 'п': 16, 'я': 17, 'г': 18, 'ь': 19, 'ы': 20, 'з': 21, 'б': 22, 'ч': 23, 'й': 24, 'ж': 25, 'ш': 26, 'х': 27, 'ю': 28, 'ц': 29, 'э': 30, 'щ': 31, 'ф': 32, 'ъ': 33}
(666713, 34)


In [4]:
model = Sequential()
model.add(Input((inp_chars, num_characters))) 
# при тренировке в рекуррентные модели keras подается сразу вся последовательность, поэтому в input теперь два числа. 1-длина последовательности, 2-размер OHE
model.add(SimpleRNN(128, activation='tanh')) # рекуррентный слой на 500 нейронов
model.add(Dense(num_characters, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 128)               20864     
_________________________________________________________________
dense (Dense)                (None, 34)                4386      
Total params: 25,250
Trainable params: 25,250
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X, Y, batch_size=256, epochs=10)

In [12]:
def buildPhrase(inp_str, str_len=50):
    for i in range(str_len):
        x = []
        for j in range(i, i + inp_chars):
            x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в One-Hot-encoding
        x = np.array(x)
        inp = x.reshape(1, inp_chars, num_characters)
        pred = model.predict(inp) # предсказываем OHE четвертого символа
        d = tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответв символьном представлении
        inp_str += d # дописываем строку
    return inp_str

In [13]:
res = buildPhrase("утренн")
print(res)

утренно сказал он                                       
